In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import os
import scipy.stats
import seaborn as sns
import matplotlib.pyplot as plt

INPUT_PATH = Path('../input')
OUTPUT_PATH = Path('../output')
pd.set_option('display.float_format', str)

In [2]:
%%time

transactions = pd.read_csv(INPUT_PATH / 'trxn.csv')
assets_under_management = pd.read_csv(INPUT_PATH / 'aum.csv')
balance = pd.read_csv(INPUT_PATH / 'balance.csv')
client = pd.read_csv(INPUT_PATH / 'client.csv')
campaigns = pd.read_csv(INPUT_PATH / 'com.csv')
deals = pd.read_csv(INPUT_PATH / 'deals.csv')
dict_merchant_category_code = pd.read_csv(INPUT_PATH / 'dict_mcc.csv')
payments = pd.read_csv(INPUT_PATH / 'payments.csv')
funnel = pd.read_csv(INPUT_PATH / 'funnel.csv')
appl = pd.read_csv(INPUT_PATH / 'appl.csv')

data = funnel.copy()

/opt/conda/lib/python3.7/site-packages/decorator.py:231: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  return caller(func, *(extras + args), **kw)


CPU times: user 7.18 s, sys: 485 ms, total: 7.66 s
Wall time: 7.66 s


In [3]:
def get_feature_diff(df, col_name1, col_name2, fill_na_value):
    return data['client_id'].map(df.groupby('client_id')[col_name1].sum() - df.groupby('client_id')[col_name2].sum()).fillna(fill_na_value)


def get_feature_rate(df, col_name1, col_name2, fill_na_value):
    return data['client_id'].map(df.groupby('client_id')[col_name1].sum() / (df.groupby('client_id')[col_name2].sum() + 1e-12)).fillna(fill_na_value)

In [4]:
campaigns.isnull().sum()

client_id            0
agr_flg              0
otkaz                0
dumaet               0
ring_up_flg          0
not_ring_up_flg      0
count_comm           0
month_end_dt         0
channel              0
prod               180
dtype: int64

In [5]:
data['diff_campaigns_otkaz-agr_flg'] = get_feature_diff(campaigns, 'otkaz', 'agr_flg', -999)
data['diff_campaigns_otkaz-agr_flg'].value_counts()

0.0       7551
-999.0    4982
1.0       4653
2.0       2264
3.0        943
-1.0       591
4.0        318
-2.0        98
5.0         61
-3.0        20
6.0          9
-4.0         6
-5.0         1
7.0          1
Name: diff_campaigns_otkaz-agr_flg, dtype: int64

In [6]:
data['rate_campaigns_otkaz-count_comm'] = get_feature_rate(campaigns, 'otkaz', 'count_comm', -999)
data['rate_campaigns_otkaz-count_comm'].value_counts()

0.0                     7710
-999.0                  4982
0.0624999999999961       301
0.08333333333332639      299
0.12499999999998437      293
                        ... 
0.0299999999999997         1
0.2499999999999844         1
0.07317073170731618        1
0.030612244897958874       1
0.020408163265305916       1
Name: rate_campaigns_otkaz-count_comm, Length: 224, dtype: int64

In [7]:
data['rate_campaigns_agr_flg-count_comm'] = get_feature_rate(campaigns, 'agr_flg', 'count_comm', -999)
data['rate_campaigns_agr_flg-count_comm'].value_counts()

0.0                     14916
-999.0                   4982
0.03571428571428444        77
0.03124999999999902        73
0.049999999999997505       70
                        ...  
0.04411764705882289         1
0.026315789473683866        1
0.027027027027026664        1
0.499999999999875           1
0.025641025641025314        1
Name: rate_campaigns_agr_flg-count_comm, Length: 123, dtype: int64

In [8]:
data['rate_campaigns_not_ring_up_flg-count_comm'] = get_feature_rate(campaigns, 'not_ring_up_flg', 'count_comm', -999)
data['rate_campaigns_not_ring_up_flg-count_comm'].value_counts()

0.0                    10446
-999.0                  4982
0.12499999999999609      125
0.11111111111110802      118
0.19999999999999002      117
                       ...  
0.8235294117646978         1
0.19230769230769046        1
0.3111111111111077         1
0.624999999999987          1
0.4390243902438971         1
Name: rate_campaigns_not_ring_up_flg-count_comm, Length: 560, dtype: int64

In [9]:
data['rate_campaigns_ring_up_flg-count_comm'] = get_feature_rate(campaigns, 'ring_up_flg', 'count_comm', -999)
data['rate_campaigns_ring_up_flg-count_comm'].value_counts()

-999.0                  4982
0.0                     3916
0.4999999999999375       453
0.33333333333330556      383
0.2499999999999844       345
                        ... 
0.039999999999999605       1
0.1960784313725471         1
0.18421052631578705        1
0.03921568627450942        1
0.11627906976744051        1
Name: rate_campaigns_ring_up_flg-count_comm, Length: 434, dtype: int64